# Creation of benchmark N-body NFW MW-type halo via sampling.

## Previous method.

In [ ]:
from shared.preface import *
import shared.functions as fct

PRE = PRE(
    sim='L025N752', 
    z0_snap=36, z4_snap=13, DM_lim=1000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=20, thetas=20, vels=200,
    pre_CPUs=128, sim_CPUs=128, mem_lim_GB=224
)

# Determine number of DM particles to sample based on virial mass.
DM_mass = 1437874.875*Msun  # from 25 Mpc box
num_DM_MW = math.floor(Mvir_MW / DM_mass)  # ~1.4 mio.
num_DM_box = 235953  # a halo from sim box

Mvir_box = num_DM_box*DM_mass

def halo_sample(z, snap, Mvir_init, R_s_init, rho0_init, DM_mass_Msun, out_dir):

    # R_200 of MW-type halo at redshift 0.
    R_200_MW = (Mvir_MW / (200 * fct.rho_crit(0) * 4.*Pi/3.))**(1./3.)
    
    # Determine R_200 and c_200 and scale radius at redshift z.
    R_200 = (Mvir_init / (200 * fct.rho_crit(z) * 4.*Pi/3.))**(1./3.)
    c_200 = fct.c_vir(z, Mvir_init, R_200_MW, R_s_init)

    # Calculate virial mass at redshift z and with it number of DM particles.
    f_200 = np.log(1+c_200) - (c_200/(1+c_200))
    M_200 = 4*Pi*rho0_init*Rs_MW**3 * f_200
    num_DM = math.floor(M_200/DM_mass_Msun)

    # Construct projection function.
    def Proj(r, r_s, f_norm):
        x = r/r_s
        f_r = np.log(1+x) - (x/(1+x))
        return f_r/f_norm

    # Construct inverse function. Needs to be without numerical units.
    invf = inversefunc(Proj, args=(Rs_MW/kpc, f_200))  

    # Sample uniformly between [0,1] and project to corresponding radius.
    sample = np.sort(np.random.uniform(size=num_DM))
    r_sample = invf(sample)

    # Sample for angles and convert to cartesian coordinates.
    phis = np.random.uniform(0, 2*Pi, num_DM)  # uniform [0,2pi)
    cos_thetas = 2.*np.random.uniform(0, 1, num_DM) - 1  # uniform [-1,1)

    # Convert to cartesian coordinates.
    x = r_sample*np.cos(phis)*np.sqrt(1. - cos_thetas**2)
    y = r_sample*np.sin(phis)*np.sqrt(1. - cos_thetas**2)
    z = r_sample*cos_thetas
    coords = np.column_stack((x,y,z))

    np.save(f'{out_dir}/benchmark_halo_snap_{snap}.npy', coords)

## Inverting procedure for each redshift (multiprocess), with commah

In [ ]:
from shared.preface import *
import shared.functions as fct

PRE = PRE(
    sim='L025N752', 
    z0_snap=36, z4_snap=13, DM_lim=10000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=10, thetas=10, vels=100,
    pre_CPUs=128, sim_CPUs=128, mem_lim_GB=224
)

In [ ]:
import commah

def halo_sample_z(z, snap, Mvir_z0, out_dir):

    # Get the DM halo mass (and the number of DM particles for sample).
    commah_output = commah.run('WMAP9', zi=0, Mi=Mvir_z0, z=z)
    Mz = commah_output['Mz'][0,0]*Msun
    num_DM = math.floor(Mz / PRE.DM_SIM_MASS)

    # Get the concentration of the halo.
    c_200 = commah_output['c'][0,0]

    # Calculate R_200 and R_s ("virial" radius and scale radius).
    R_200 = np.power(Mz / (200*fct.rho_crit(z)*4/3*Pi), 1./3.)
    R_s = R_200 / c_200

    # Construct projection function.
    def Proj(r, r_s, norm):
        x = r/r_s
        return (np.log(1+x) - (x/(1+x)))/norm

    # Construct inverse function. Needs to be without numerical units.
    f_200 = np.log(1+c_200) - (c_200/(1+c_200))
    invf = inversefunc(Proj, args=(R_s/kpc, f_200))  

    # Sample uniformly between [0,1] and project to corresponding radius.
    sample = np.sort(np.random.uniform(size=num_DM))
    r_sample = invf(sample)

    # Sample for angles and convert to cartesian coordinates.
    phis = np.random.uniform(0, 2*Pi, num_DM)  # uniform [0,2pi)
    cos_thetas = 2.*np.random.uniform(0, 1, num_DM) - 1  # uniform [-1,1)

    # Convert to cartesian coordinates.
    x = r_sample*np.cos(phis)*np.sqrt(1. - cos_thetas**2)
    y = r_sample*np.sin(phis)*np.sqrt(1. - cos_thetas**2)
    z = r_sample*cos_thetas
    coords = np.column_stack((x,y,z))

    np.save(f'{out_dir}/benchmark_halo_snap_{snap}.npy', coords)
    # return coords

# for zed, snap in zip(PRE.ZEDS_SNAPS[::-1], PRE.NUMS_SNAPS[::-1]):
#     halo_sample_z(zed, snap, Mvir_MW/Msun, PRE.OUT_DIR)

In [ ]:
with ProcessPoolExecutor(24) as ex:
    ex.map(
        halo_sample_z, PRE.ZEDS_SNAPS[::-1], PRE.NUMS_SNAPS[::-1],
        repeat(Mvir_MW/Msun), repeat(PRE.OUT_DIR)
    )

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for snap in PRE.NUMS_SNAPS[::-1][[0, 10]]:
    coords = np.load(f'{PRE.OUT_DIR}/benchmark_halo_snap_{snap}.npy')
    print(len(coords))
    ss = 100
    x, y, z = coords[:,0][::ss], coords[:,1][::ss], coords[:,2][::ss]
    ax.scatter(x,y,z, alpha=1, s=0.05)
plt.show()

## Density profile check.

In [ ]:
# Bin centers in kpc.
radial_bins = 10**np.arange(0, 5, 0.1)
centers = bin_centers(radial_bins)

# Calculate density for each bin center.
density = analyse_halo(DM_mass, coords)

fig = plt.figure()

# Density profile of sampled halo.
u1, u2 = kpc, Msun/kpc**3
r, d = centers*u1, density*u2
plt.loglog(r/u1, d/u2, c='blue', label=f'sampled halo')

# NFW density profile with parameters of sampled halo.
rho_NFW = scale_density_NFW(c=c200)
NFW = fct.NFW_profile(r, rho_NFW, r_s*kpc)
plt.loglog(r/u1, NFW/u2, c='black', ls='-.', label='NFW')

plt.title('Benchmark halo density profile')
plt.xlabel('radius from halo center [kpc]')
plt.ylabel('density in [Msun/kpc^3]')
plt.xlim(np.min(r/kpc), 1.3*Rvir)
plt.ylim(1e1, 1e9)
plt.legend()
plt.show()